# 畳み込み演算とプーリングの数理

このノートブックでは、CNNの核心である**畳み込み演算**と**プーリング**を数理的に理解します。

## 目次
1. 積和演算とは
2. 畳み込みフィルタの適用
3. ストライドとバイアス
4. Max Pooling
5. 実装例

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap

## 1. 積和演算とは

**積和演算**（せきわえんざん）は、畳み込み層の基本操作です。

入力データと畳み込みフィルタの**同じ位置同士**の値を掛け算し、**総和**を計算します。

### 数式
$$\text{出力} = \sum_{i}\sum_{j} (\text{入力}_{i,j} \times \text{フィルタ}_{i,j})$$

In [ ]:
# 図3.5, 3.6: 入力データと畳み込みフィルタ

# 入力データ (3×3)
input_data = np.array([
    [1, 1, 0],
    [0, 2, 1],
    [3, 0, 1]
])

# 畳み込みフィルタ (2×2)
kernel = np.array([
    [1, 0],
    [0, 2]
])

print("入力データ (3×3):")
print(input_data)
print("\n畳み込みフィルタ (2×2):")
print(kernel)

In [ ]:
# 積和演算の視覚化（左上2×2領域の場合）

fig, axes = plt.subplots(1, 4, figsize=(14, 4))

# 入力データ（左上2×2をハイライト）
ax = axes[0]
ax.imshow(input_data, cmap='Blues', vmin=0, vmax=3)
for i in range(3):
    for j in range(3):
        ax.text(j, i, str(input_data[i, j]), ha='center', va='center', fontsize=16)
rect = Rectangle((-0.5, -0.5), 2, 2, fill=False, edgecolor='red', linewidth=3)
ax.add_patch(rect)
ax.set_title('入力データ\n(赤枠が対象領域)', fontsize=11)
ax.axis('off')

# フィルタ
ax = axes[1]
ax.imshow(kernel, cmap='Oranges', vmin=0, vmax=2)
for i in range(2):
    for j in range(2):
        ax.text(j, i, str(kernel[i, j]), ha='center', va='center', fontsize=16)
ax.set_title('畳み込みフィルタ', fontsize=11)
ax.axis('off')

# 計算過程
ax = axes[2]
ax.text(0.5, 0.8, '積和演算:', ha='center', fontsize=12, fontweight='bold')
ax.text(0.5, 0.6, '(左上) 1 × 1 = 1', ha='center', fontsize=11)
ax.text(0.5, 0.45, '(右上) 1 × 0 = 0', ha='center', fontsize=11)
ax.text(0.5, 0.3, '(左下) 0 × 0 = 0', ha='center', fontsize=11)
ax.text(0.5, 0.15, '(右下) 2 × 2 = 4', ha='center', fontsize=11)
ax.text(0.5, 0.0, '―――――――', ha='center', fontsize=11)
ax.text(0.5, -0.15, '合計 = 5', ha='center', fontsize=14, fontweight='bold', color='red')
ax.set_xlim(0, 1)
ax.set_ylim(-0.3, 1)
ax.axis('off')

# 結果
ax = axes[3]
result = np.array([[5]])
ax.imshow(result, cmap='Greens', vmin=0, vmax=10)
ax.text(0, 0, '5', ha='center', va='center', fontsize=20, fontweight='bold')
ax.set_title('出力値', fontsize=11)
ax.axis('off')

plt.suptitle('図3.6: 畳み込みフィルタによる積和演算', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 2. 畳み込みフィルタの適用

畳み込みでは、フィルタを入力データ上で**スライド**させながら、各位置で積和演算を行います。

In [ ]:
def convolution_2d(input_data, kernel, stride=1):
    """2D畳み込み演算を行う"""
    h, w = input_data.shape
    kh, kw = kernel.shape
    
    # 出力サイズを計算
    out_h = (h - kh) // stride + 1
    out_w = (w - kw) // stride + 1
    
    output = np.zeros((out_h, out_w))
    
    for i in range(out_h):
        for j in range(out_w):
            # 対象領域を切り出し
            region = input_data[i*stride:i*stride+kh, j*stride:j*stride+kw]
            # 積和演算
            output[i, j] = np.sum(region * kernel)
    
    return output

# 畳み込み実行
output = convolution_2d(input_data, kernel, stride=1)
print("畳み込み結果 (2×2):")
print(output)

In [ ]:
# 図3.7: ストライド幅1での全位置の積和演算を視覚化

fig, axes = plt.subplots(2, 4, figsize=(14, 7))

positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
results = [5, 3, 0, 4]

for idx, (pos, result) in enumerate(zip(positions, results)):
    row, col = pos
    
    # 入力データ（対象領域をハイライト）
    ax = axes[0, idx]
    ax.imshow(input_data, cmap='Blues', vmin=0, vmax=3)
    for i in range(3):
        for j in range(3):
            ax.text(j, i, str(input_data[i, j]), ha='center', va='center', fontsize=14)
    rect = Rectangle((col-0.5, row-0.5), 2, 2, fill=False, edgecolor='red', linewidth=3)
    ax.add_patch(rect)
    ax.set_title(f'位置 ({row}, {col})', fontsize=10)
    ax.axis('off')
    
    # 計算結果
    ax = axes[1, idx]
    region = input_data[row:row+2, col:col+2]
    calc = ' + '.join([f'{region[i,j]}×{kernel[i,j]}' for i in range(2) for j in range(2)])
    ax.text(0.5, 0.7, f'{calc}', ha='center', fontsize=9)
    ax.text(0.5, 0.3, f'= {result}', ha='center', fontsize=16, fontweight='bold', color='red')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

plt.suptitle('図3.7: ストライド幅1での積和演算（4つの位置）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n出力（特徴マップ）:")
print(output)

## 3. ストライドとバイアス

### ストライド（Stride）
フィルタを移動させる幅のこと。
- stride=1: 1ピクセルずつ移動
- stride=2: 2ピクセルずつ移動（出力サイズが半分に）

### バイアス（Bias）
積和演算の結果に加算する定数項。ニューラルネットワークの学習で調整されるパラメータの一つ。

In [ ]:
# バイアス項の加算
bias = 1

output_with_bias = output + bias

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# 畳み込み結果
ax = axes[0]
ax.imshow(output, cmap='Greens', vmin=0, vmax=10)
for i in range(2):
    for j in range(2):
        ax.text(j, i, str(int(output[i, j])), ha='center', va='center', fontsize=16)
ax.set_title('畳み込み結果', fontsize=12)
ax.axis('off')

# +
ax = axes[1]
ax.text(0.5, 0.5, f'+ bias({bias})', ha='center', va='center', fontsize=20)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

# バイアス加算後
ax = axes[2]
ax.imshow(output_with_bias, cmap='Greens', vmin=0, vmax=10)
for i in range(2):
    for j in range(2):
        ax.text(j, i, str(int(output_with_bias[i, j])), ha='center', va='center', fontsize=16)
ax.set_title('バイアス加算後（最終出力）', fontsize=12)
ax.axis('off')

plt.suptitle('バイアス項の加算', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"畳み込み結果:\n{output}")
print(f"\nバイアス加算後:\n{output_with_bias}")

## 4. Max Pooling

**Max Pooling**は、指定した範囲内の**最大値**を選ぶことでデータサイズを圧縮する処理です。

### 目的
- データサイズの削減（計算量削減）
- 重要な特徴量（大きい値）を残す
- 位置の微小なズレに対するロバスト性

In [ ]:
def max_pooling_2d(input_data, pool_size=2, stride=2):
    """Max Poolingを行う"""
    h, w = input_data.shape
    
    out_h = (h - pool_size) // stride + 1
    out_w = (w - pool_size) // stride + 1
    
    output = np.zeros((out_h, out_w))
    
    for i in range(out_h):
        for j in range(out_w):
            region = input_data[i*stride:i*stride+pool_size, j*stride:j*stride+pool_size]
            output[i, j] = np.max(region)  # 最大値を選択
    
    return output

# 図3.8: Max Poolingの例
pool_input = np.array([
    [9,  10, 18, 10],
    [12, 24, 13, 12],
    [13, 19, 22, 19],
    [15, 6,  15, 16]
])

pool_output = max_pooling_2d(pool_input, pool_size=2, stride=2)

print("Max Pooling入力 (4×4):")
print(pool_input)
print("\nMax Pooling出力 (2×2):")
print(pool_output)

In [ ]:
# 図3.8: Max Poolingの視覚化

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

# 入力データ（4つの領域を色分け）
ax = axes[0]
ax.imshow(pool_input, cmap='Blues', vmin=0, vmax=30)
for i in range(4):
    for j in range(4):
        ax.text(j, i, str(pool_input[i, j]), ha='center', va='center', fontsize=12)
# 4つの領域を枠で囲む
colors = ['red', 'green', 'orange', 'purple']
for idx, (r, c) in enumerate([(0, 0), (0, 2), (2, 0), (2, 2)]):
    rect = Rectangle((c-0.5, r-0.5), 2, 2, fill=False, edgecolor=colors[idx], linewidth=3)
    ax.add_patch(rect)
ax.set_title('入力 (4×4)\n各色の領域からmax値を取得', fontsize=11)
ax.axis('off')

# 矢印
ax = axes[1]
ax.text(0.5, 0.7, 'Max Pooling', ha='center', fontsize=14, fontweight='bold')
ax.text(0.5, 0.5, '2×2, stride=2', ha='center', fontsize=12)
ax.annotate('', xy=(0.8, 0.3), xytext=(0.2, 0.3),
            arrowprops=dict(arrowstyle='->', lw=3))
ax.text(0.5, 0.1, '各領域の最大値を選択', ha='center', fontsize=10)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

# 出力
ax = axes[2]
ax.imshow(pool_output, cmap='Greens', vmin=0, vmax=30)
for i in range(2):
    for j in range(2):
        ax.text(j, i, str(int(pool_output[i, j])), ha='center', va='center', 
                fontsize=16, fontweight='bold')
ax.set_title('出力 (2×2)', fontsize=11)
ax.axis('off')

plt.suptitle('図3.8: Max Poolingの実行', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 各領域の計算を表示
print("各領域のMax値:")
print(f"  左上: max(9, 10, 12, 24) = 24")
print(f"  右上: max(18, 10, 13, 12) = 18")
print(f"  左下: max(13, 19, 15, 6) = 19")
print(f"  右下: max(22, 19, 15, 16) = 22")

## 5. 実装例：PyTorchでの畳み込みとプーリング

PyTorchを使った実装例を見てみましょう。

In [ ]:
import torch
import torch.nn as nn

# 入力データをPyTorchテンソルに変換 (バッチサイズ, チャンネル, 高さ, 幅)
input_tensor = torch.tensor(input_data, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
print(f"入力テンソルの形状: {input_tensor.shape}")

# 畳み込み層を定義
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=1, bias=False)

# フィルタの重みを手動で設定
with torch.no_grad():
    conv.weight = nn.Parameter(torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0))

# 畳み込み実行
conv_output = conv(input_tensor)
print(f"\n畳み込み出力の形状: {conv_output.shape}")
print(f"畳み込み出力:\n{conv_output.squeeze().detach().numpy()}")

In [ ]:
# Max Poolingの実行
pool_input_tensor = torch.tensor(pool_input, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
pool_output_tensor = max_pool(pool_input_tensor)

print(f"Max Pooling入力の形状: {pool_input_tensor.shape}")
print(f"Max Pooling出力の形状: {pool_output_tensor.shape}")
print(f"\nMax Pooling出力:\n{pool_output_tensor.squeeze().numpy()}")

## まとめ

### 畳み込み層
- **積和演算**: 入力とフィルタの同じ位置を掛けて総和
- **ストライド**: フィルタの移動幅
- **バイアス**: 出力に加算する定数
- **出力サイズ**: `(入力サイズ - フィルタサイズ) / ストライド + 1`

### プーリング層（Max Pooling）
- 領域内の**最大値**を選択
- データサイズを圧縮
- 重要な特徴を保持

## 次のステップ

次のノートブックでは、**誤差逆伝播法**と**学習のメカニズム**について学びます。